In [1]:
from modeling.compute import Var, create_vars
from modeling.arghandling import Encoder, encode
from foundations.functionals import encode_sympy
from foundations.functional_noobj import eliminate_vars
from foundations.functional_noobj import feed_forward
from foundations.functional_noobj import partial_function
from foundations.functional_noobj import residual_computer
from foundations.functional_noobj import residual_solver_determined
from foundations.functional_noobj import residual_solver
from foundations.functional_noobj import concatenate_residuals
from foundations.functional_noobj import optimizer_solver
from foundations.functional_noobj import external_encoder
from sympy import exp
import numpy as np

# Aircraft

In [2]:
A,V,S,W,Ww,CL,Re,Cf,CD,D = create_vars('A V S W W_w C_L Re C_f C_D D')
rho,mu,k,t,e,CDA,SwetS,W0 = 1.23, 1.78e-5, 1.2, 0.12, 0.96, 0.0306, 2.05, 4940

F1 = encode_sympy(2*W/(rho*V**2*S), CL)
F2 = encode_sympy(rho*V/mu*(S/A)**(1/2), Re)
F3 = encode_sympy(0.074/Re**0.2, Cf)
F4 = encode_sympy(CDA/S+k*Cf*SwetS+CL**2/(np.pi*A*e), CD)
F5 = encode_sympy(0.5*rho*V**2*CD*S, D)
F6 = encode_sympy(45.42*S+8.71e-5*2.5/t*A**(3/2)*S**(1/2)*(W0*W)**(1/2), Ww)
R1 = encode_sympy(W0+Ww-W)

xin = {A:18.16, S: 5.255, V: 49.18}

In [3]:
F = feed_forward([F1, F2, F3, F4, F5, F6])
R2 = eliminate_vars(R1, F)
#R = concatenate_residuals([R1, R2])
RS = residual_solver(R2, Encoder((W,)), x_initial={W:7000})

In [4]:
MDF = feed_forward([RS, F])
MDF.dict_in_dict_out(xin, cleanup=True)

{W: 7082.846459912354,
 C_L: 0.906114212539708,
 Re: 1828108.936362036,
 C_f: 0.004138388946405071,
 C_D: 0.03099440488006809,
 D: 242.27476828398164,
 W_w: 2142.8464599123567}

In [18]:
MDF.dict_in_dict_out(xopt,cleanup=True)

{W: 7083.575373937885,
 C_L: 0.90600653382041,
 Re: 1828150.8796346504,
 C_f: 0.004138369956816118,
 C_D: 0.030987531620324762,
 D: 242.27476038088258,
 W_w: 2143.575373937888}

In [5]:
obj = eliminate_vars(encode_sympy(D), MDF)

In [7]:
xin = {A:10,S:10,V:10,W:7000}
external_encoder(obj, Encoder((A,S,V,W)))(np.array([10,10,10,7000]))
external_encoder(R2, Encoder((A,S,V,W)))(np.array([10,10,10,7000]))

array([-538.738611])

In [31]:
R2.flat_in_only(encode(xin, R2.encoder.order, flatten=True))

array([-538.738611])

In [22]:
obj.flat_in_only(encode(xin, obj.encoder.order, flatten=True)),


array([], dtype=float64)

In [8]:
OPT = optimizer_solver(obj, eqs=(R2,), bounds={
    A:(0.1, 100),S:(0.1, 100),V:(0.1, 100),W:(0.1, 1e6)})

In [10]:
xopt = OPT.dict_in_dict_out(x0={A:10,S:10,V:10,W:1000})

c:\Users\johan\miniconda3\envs\minimdo\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


In [27]:
xin = {A:18.16, S: 5.255, V: 49.18}
y = RS.dict_in_dict_out(xin, 
                    x_initial={W:1000})

In [30]:
F.dict_in_dict_out({**xin, **y}, 
                   cleanup=True)

{C_L: 0.9061142125397086,
 Re: 1828108.936362036,
 C_f: 0.004138388946405071,
 C_D: 0.030994404880068112,
 D: 242.27476828398179,
 W_w: 2142.846459912357}

# Sellar

In [2]:
z1,z2,x,y2 = Var('z_1'), Var('z_2'), Var('x'), Var('y_2')
a = Var('a')
y1 = Var('y_1')

In [3]:
F1 = encode_sympy(z1+z2-0.2*y2, a)
F2 = encode_sympy(x**2+a, y1)

In [4]:
Fnew = eliminate_vars(F2, F1)

In [5]:
Fnew.dict_in_dict_out({z1:1, z2:1, y2:1, x:1})

{y_1: 2.8}

In [6]:
F = feed_forward([F2, F1])

In [7]:
F.dict_in_dict_out({z1:1, z2:1, y2:1, x:1, a:1.8})

{y_1: 2.8, a: 1.8}

In [8]:
P = partial_function(F1, Encoder([z1,z2]))
PF = P.dict_in_only({z1:1, z2:1})

In [9]:
PF.dict_in_dict_out({y2:1})

{a: 1.8}

In [10]:
R1 = encode_sympy(z1+z2-0.2*y2-a)
R2 = encode_sympy(x**2+a-y1)

In [11]:
RS = residual_solver(R1, Encoder((z1,)))

In [12]:
RS

(z_1,) <- (y_2, a, z_2)

In [13]:
random_generator = np.random.default_rng(seed=2023).random
RS.dict_in_dict_out({y2:1, z2:1, a:1},
                    random_generator=random_generator)

{z_1: 0.1999999999999999}

In [14]:
F1.dict_in_dict_out({z1:0.2, z2:1, y2:1})

{a: 1.0}

In [7]:
F.dict_in_dict_out({A:18.16, S: 5.255, V: 49.18, W:7083.65}, cleanup=True)

{C_L: 0.906217010065799,
 Re: 1828108.936362036,
 C_f: 0.004138388946405071,
 C_D: 0.030997806480403657,
 D: 242.3013576615252,
 W_w: 2142.95446938366}